# １导入必要的库

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import lightgbm as lgb
import pandas as pd
import numpy as np
import gc
from tqdm import tqdm_notebook as tqdm
import copy


# 2 载入必要的文件

In [2]:
train=pd.read_csv('../data/meinian_round1_train_20180408.csv',sep=',',encoding='gbk')
test=pd.read_csv('../data/meinian_round1_test_b_20180505.csv',sep=',',encoding='gbk')
data_part1=pd.read_csv('../data/meinian_round1_data_part1_20180408.zip',sep='$',encoding='utf-8')
data_part2=pd.read_csv('../data/meinian_round1_data_part2_20180408.zip',sep='$',encoding='utf-8')
physical_data = data_part1.append(data_part2)

# 3 基本数据处理

In [3]:
physical_data['field_results'] = physical_data['field_results'].astype('str')
physical_data = pd.pivot_table(physical_data,index='vid',columns='table_id',values='field_results',aggfunc=lambda x: '_'.join(x))
physical_data.to_pickle('../data/physical_data.pkl')
index = np.save('../data/index.npy',physical_data.index)

# 4 EDA,及一些对数据的人为分析，后续看不懂的地方都在这一步完成．

# 5 数据清洗及特征提取

In [4]:
# 增加项目数作为特征
physical_data.reset_index(drop=True,inplace=True)
physical_data['subject_num'] = 2795-physical_data.isnull().sum(axis=1)
column_name = physical_data.columns
drop_column = []
valid_column = []
valid_df = pd.DataFrame()
for f in range(len(column_name)):
    if physical_data.iloc[:,f].isnull().sum()>55000:
        drop_column.append(f)
    else:
        valid_column.append(f)
print('%d 列缺失值超过阈值'%len(drop_column))
new_df = pd.DataFrame()
for i in valid_column:
    new_df[column_name[i]] = physical_data.iloc[:,i]
print(new_df.shape)

# 统计特征类型数目
physical_data = new_df
column_name = list(physical_data.columns.values)
numerical_feature = []
for item in tqdm(column_name):
    try:
        physical_data[item] = physical_data[item].astype('float16')
        numerical_feature.append(item)
    except:
        continue
str_feature = []
for item in column_name:
    if item not in numerical_feature:
        str_feature.append(item)
print('数值特征数目%d'% (len(numerical_feature)))
print('字符特征数目%d'%(len(str_feature)))

# clean the data
#数据预处理之删除所有空格，替换ＩI,替换'阴性'为'-',
#[用'_'分割一模一样则合并,有nan则删除，有数字和其他则保留数字，都是数字取平均值,doushi]，从<0.6提取关键信息
str_feature = physical_data.columns
for j in tqdm(str_feature):
    row = []
    for i in range(len(physical_data)):
        value = physical_data.loc[i,j]
        value = str(value).strip()
        if value in ['nan','None']:
            row.append(np.NaN)
        else:
            try:
                row.append(float(value))
            except:
                if '_' in value:
                    value_list = []
                    for sub_value in value.split('_'):
                        # 去空格
                        sub_value = str(sub_value).strip()
                        # 去空值
                        if sub_value in ['nan','None']:
                            continue
                        else:
                            try:
                                value_list.append(float(value))
                            except:
                                if '未检' in sub_value or '弃查' in sub_value or '未查' in sub_value or '未检查'in sub_value:
                                    continue
                                else:
                                    sub_value = sub_value.replace('阴性','-')
                                    sub_value = sub_value.replace('阳性','+')
                                    sub_value = sub_value.replace('偏阳','+-')
                                    sub_value = sub_value.replace('P','+')
                                    sub_value = sub_value.replace('p','+')
                                    sub_value = sub_value.replace('N','-')
                                    sub_value = sub_value.replace('IV','4')
                                    sub_value = sub_value.replace('V','5')
                                    sub_value = sub_value.replace('III','3')
                                    sub_value = sub_value.replace('II','2')
                                    sub_value = sub_value.replace('I','1')
                                    sub_value = sub_value.replace('。','.')
                                    sub_value = sub_value.strip('.')
                                    sub_value = sub_value.strip('°')
                                    sub_value = sub_value.strip('%')
                                    sub_value = sub_value.strip('度')
                                    sub_value = sub_value.strip('mmol/L')
                                    sub_value = sub_value.strip('自述不查')
                                    sub_value = sub_value.strip('详见报告单')
                                    sub_value = sub_value.strip('/')
                                    sub_value = sub_value.strip('标本已退检')
                                    sub_value = sub_value.strip()
                                    sub_value = sub_value.replace('..','.')
                                    sub_value = sub_value.replace('＜','<')
                                    sub_value = sub_value.replace('＞','>')
                                    sub_value = sub_value.replace('>','')
                                    sub_value = sub_value.replace('<','')
                                    sub_value = sub_value.replace('＋','+')
                                    sub_value = sub_value.replace('－','-')
                                    sub_value = sub_value.replace('iv','４')
                                    sub_value = sub_value.replace('iii','3')
                                    sub_value = sub_value.replace('ii','2')
                                    sub_value = sub_value.replace('i','1')
                                    sub_value = sub_value.replace('Ⅲ','3')
                                    sub_value = sub_value.replace('Ⅱ','2')
                                    sub_value = sub_value.replace('Ⅰ','1')
                                    sub_value = sub_value.replace('Ⅳ','4')
                                    sub_value = sub_value.replace('中度','2')
                                value_list.append(sub_value)
                    if value_list == []:
                        row.append(np.NaN)
                    else:
                        # 去重
                        value_list = list(set(value_list))
                        # 都是数字，取平均值
                        try:
                            row.append(np.mean([float(x) for x in value_list]))
                        except:
                            flag = False
                            for sth in value_list:
                                try:
                                    float(sth)
                                    row.append(sth)
                                    flag = True
                                    break
                                except:
                                    continue
                            
                            if flag:
                                pass
                            else:
                                value_list = '_'.join(value_list)
                                row.append(value_list)
                else:
                    sub_value = value
                    if '未检' in sub_value or '弃查' in sub_value or '未查' in sub_value or '未检查'in sub_value:
                        row.append(np.NaN)
                    else:
                        sub_value = sub_value.replace('阴性','-')
                        sub_value = sub_value.replace('阳性','+')
                        sub_value = sub_value.replace('偏阳','+-')
                        sub_value = sub_value.replace('P','+')
                        sub_value = sub_value.replace('p','+')
                        sub_value = sub_value.replace('N','-')
                        sub_value = sub_value.replace('IV','4')
                        sub_value = sub_value.replace('V','5')
                        sub_value = sub_value.replace('III','3')
                        sub_value = sub_value.replace('II','2')
                        sub_value = sub_value.replace('I','1')
                        sub_value = sub_value.replace('。','.')
                        sub_value = sub_value.strip('.')
                        sub_value = sub_value.strip('°')
                        sub_value = sub_value.strip('%')
                        sub_value = sub_value.strip('度')
                        sub_value = sub_value.strip('mmol/L')
                        sub_value = sub_value.strip('自述不查')
                        sub_value = sub_value.strip('详见报告单')
                        sub_value = sub_value.strip('/')
                        sub_value = sub_value.strip('标本已退检')
                        sub_value = sub_value.strip()
                        sub_value = sub_value.replace('..','.')
                        sub_value = sub_value.replace('＜','<')
                        sub_value = sub_value.replace('＞','>')
                        sub_value = sub_value.replace('>','')
                        sub_value = sub_value.replace('<','')
                        sub_value = sub_value.replace('＋','+')
                        sub_value = sub_value.replace('－','-')
                        sub_value = sub_value.replace('iv','４')
                        sub_value = sub_value.replace('iii','3')
                        sub_value = sub_value.replace('ii','2')
                        sub_value = sub_value.replace('i','1')
                        sub_value = sub_value.replace('Ⅲ','3')
                        sub_value = sub_value.replace('Ⅱ','2')
                        sub_value = sub_value.replace('Ⅰ','1')
                        sub_value = sub_value.replace('Ⅳ','4')
                        sub_value = sub_value.replace('中度','2')
                        row.append(sub_value)

    physical_data[j] = row
    
    

2388 列缺失值超过阈值
(57298, 408)


HBox(children=(IntProgress(value=0, max=408), HTML(value='')))


数值特征数目97
字符特征数目311


HBox(children=(IntProgress(value=0, max=408), HTML(value='')))

# 6 特征转换为数值

In [5]:
# 手动确定一些肉眼观测过的列为数值列和无关列
customized_num_column = ['100005', '100007', '10002', '10003' ,'1107', '1115', '1117', '1127', '1345',
 '1814', '1845', '190', '2174', '2403', '2404', '2405', '2406', '2420', '269007',
 '269008', '269009', '269010', '269011', '269014', '269015', '269016', '269017',
 '269018', '269019', '269020', '269021', '269022', '269023', '269024', '269025',
 '315', '316', '317', '319', '33', '34', '37', '39', '669001', '809009', '979004',
 '979005', '979006', '979008', '979009', '979011', '979013', '979014', '979015',
 '979016', '979017', '979018', '979019', '979020', '979021', '979022', '979023',
 '0422', '0423', '0977', '1103', '1106', '1303', '1304', '1305', '1308' ,'1313',
 '1314', '1315', '1316', '1330', '139', '143', '1474', '155', '1840', '1850', '191',
 '192', '2231', '2501', '269003', '269013', '300036']
customized_drop_column = ['0121', '0122', '0123', '0202', '0203', '0206', '0207' ,'0208', '0209', '0210',
 '0212', '0215', '0216', '0225', '0403', '0405', '0406', '0413', '0420', '0421',
 '0425', '0426', '0427', '0428', '0429', '0430', '0434', '0435', '0436', '0501',
 '0503', '0509', '0516', '0702', '0703', '0705', '0706', '0707', '0709', '0912',
 '0947', '0949', '0954', '0972', '0973', '0974', '0975', '0980', '0983', '0984',
 '0985', '0987', '100014', '1001', '1102', '1301', '1329', '1402', '31', '312', '313']


In [6]:
column_name = list(physical_data.columns.values)
numerical_feature = []
for item in tqdm(column_name):
    try:
        physical_data[item] = physical_data[item].astype('float16')
        numerical_feature.append(item)
    except:
        continue
str_feature = []
for item in column_name:
    if item not in numerical_feature:
        str_feature.append(item)
print('数值特征数目%d'% (len(numerical_feature)))
print('字符特征数目%d'%(len(str_feature)))

# 1 原始数值
new_df = pd.DataFrame()
for i in numerical_feature:
    new_df[i] = physical_data.loc[:,i]
print(new_df.shape)

# 2 新增数值
num_func = customized_num_column
num_func = [x for x in num_func if x not in numerical_feature]
for i in num_func:
    name = i
    new_column = []
    for value in physical_data.loc[:,name]:
        try:
            new_column.append(float(value))
        except:
            value = value.replace('-','')
            value = value.replace('+','')
            if '-' in value:
                try:
                    t = np.mean(map(float,value.split('-')))
                    new_column.append(t)
                    print(t)
                except:
                    new_column.append(np.NaN)
            else:
                try:
                    new_column.append(float(value))
                except:
                    new_column.append(np.NaN)
    try:
        map(float,new_column)
    except:
        print('bad feature',i)
    new_df[name] = new_column
print(new_df.shape)

negative_word = ['+','慢性','肿大','痛疼','压痛','出现异常','不正常','差','不佳','欠均匀','坏死','异常','显示不清','扩张','增大',
                '增厚','痛','轻度','重度','晚期','偏高','偏低','不清晰','畸形','先天性','瘘管','囊肿','湿疹','肥大','糖尿病','糜烂','出血','充血','咽炎','建议医院及时就诊','建议及时就诊','过长','过短','建议进一步','手术治疗','观察',
                '血肿','紊乱','增生','炎','疑似','确诊','结节','疝','糖尿病','瘤','水肿','+-''失常','建议复查']     
positive_word = ['未见明显异常','未见异常','-','正常','健康','未触及','未检出','弃查','未查','未见异','无特殊','未及','整齐',
                '齐','单字无','无重大手术','无过敏史','无叩痛','无特殊记载','未检','活动自如','阴性','良好','及格','无扩张',
                '未显示','未探及','未闻及']
negative_sign = ['无','未发现','未出现','没有','不','未见']

# ３增加等级数值
drop = customized_drop_column
numerical_feature = numerical_feature+num_func
column_name = list(physical_data.columns.values)
remain_str = [x for x in column_name if x not in numerical_feature  and x not in drop]
print('剩余字符特征%d'%len(remain_str))
huayan_feature = []
nume = 0
char = 0
for item in tqdm(remain_str):
    this_column = physical_data.loc[:,item]
    new_column = []
    num_value = 0
    grade_value = 0
    other = 0
    for value in this_column:
        value = str(value)
        if value == 'nan':
            continue
        else:
            try:
                float(value)
                num_value+=1
            except:
                if '-' in value or '+' in value:
                    grade_value+=1
                else:
                    other+=1
    total = other+num_value+grade_value
    # 转数字列
    if num_value/total>0.6:
        nume+=1
        for value in this_column:
            value = str(value)
            if value == 'nan':
                new_column.append(np.NaN)
            else:
                try:
                    float(value)
                    new_column.append(abs(float(value)))
                except:
                    if '+' in value:
                        if new_column == []:
                            c = [1]
                        else:
                            c = new_column
                        new_column.append(np.log(max(c)+1))
                    elif '-' in value:
                        if new_column == []:
                            c = [0]
                        else:
                            c = new_column
                        new_column.append(min(c))
                    else:
                        new_column.append(np.NaN)
        new_df[item] = new_column
    # 转等级列
    elif grade_value/total>0.6:
        char+=1
        huayan_feature.append(item)
        for value in this_column:
            value = str(value)
            if value == 'nan':
                new_column.append(np.NaN)
            elif value in ['少','少见','低','较低','少量']:
                new_column.append(-0.5)
            elif '多' in value or '高'in value or '强' in value:
                new_column.append(1)
            elif '+' in value or '-' in value:
                plus = value.count('+')
                minus = 0-value.count('-')*0.5
                new_column.append(plus+minus)
            else:
                new_column.append(np.NaN)
        new_df[item] = new_column

    # 转正常、异常二分类　
    else:
        positive = 0
        negative = 0
        for value in this_column:
            value = str(value)
            if value == 'nan':
                new_column.append(np.NaN)
            else:
                be_pos = True
                for word in negative_word:
                    if word in value:
                        value = value.replace(word, 'haha')
                        be_neg = True
                        for n in negative_sign:
                            if n in value:
                                be_neg = False
                                break
                        if be_neg:
                            new_column.append(1)
                            be_pos = False
                            break
                        else:
                            new_column.append(0)
                            be_pos = False
                            break
                if be_pos:
                    if '无'==value:
                        new_column.append(0)
                    else:
                        pos_sign = False
                        for word in positive_word:
                            if word in value:
                                value = value.replace(word, 'haha')
                                be_pos = True
                                for n in negative_sign:
                                    if n in value:
                                        be_pos = False
                                        break
                                if be_pos:
                                    new_column.append(0)
                                    pos_sign = True
                                    break
                                else:
                                    new_column.append(1)
                                    pos_sign = True
                                    break
                        if not pos_sign:
                            new_column.append(1)
        new_df[item] = new_column
print(new_df.shape)
print(nume,char)
# print(len(new_column)-physical_data.shape[0])
                    

HBox(children=(IntProgress(value=0, max=408), HTML(value='')))


数值特征数目170
字符特征数目238
(57298, 170)
(57298, 194)
剩余字符特征154


HBox(children=(IntProgress(value=0, max=154), HTML(value='')))


(57298, 348)
58 23


In [7]:
raw_data = pd.read_pickle('../data/physical_data.pkl')
# 加入化验指标
huayan_df = pd.DataFrame()
for item in huayan_feature:
    huayan_df[item] = raw_data.loc[:,item]
new_df['huayan']=len(huayan_feature)-huayan_df.isnull().sum(axis=1)

# temp_df
column_name = raw_data.columns
drop_column = []
valid_column = []
valid_df = pd.DataFrame()
for f in column_name:
    if raw_data.loc[:,f].isnull().sum()>56000:
        drop_column.append(f)
    else:
        valid_column.append(f)
print('%d 列缺失值超过阈值'%len(drop_column))

# temp_df
temp_df = pd.DataFrame()
str_feature_temp = [x for x in raw_data.columns if x not in new_df.columns and x in valid_column]
print(len(str_feature_temp))
for item in str_feature_temp:
    temp_df[item] = raw_data.loc[:,item]
    
# 某些特征指标
new_name = {'肝炎':['肝炎'],'高血压':['高血压','血压偏高'],'高血脂':['高血脂','血脂偏高','高尿酸'],'血管':['动脉硬化','弹性减弱'
       '动脉轻度硬化','血管狭窄','动脉重度减弱','血管堵塞']}
x = temp_df.shape[0]
y = temp_df.shape[1]
print(x)
print(new_df.shape)
for item,values in new_name.items():
    new_column = []
    for i in tqdm(range(x)):
        c = 0
        for j in range(y):
            v = raw_data.iloc[i,j]
            if isinstance(v,str):
                for sub_name in values:
                    if sub_name in v:
                        c = 1
                        break
                if c == 1:
                    break
        new_column.append(c)
    new_df[item] = new_column
           

2303 列缺失值超过阈值
145
57298
(57298, 349)


HBox(children=(IntProgress(value=0, max=57298), HTML(value='')))

HBox(children=(IntProgress(value=0, max=57298), HTML(value='')))

HBox(children=(IntProgress(value=0, max=57298), HTML(value='')))

HBox(children=(IntProgress(value=0, max=57298), HTML(value='')))

In [8]:
new_df.to_pickle('../data/train_data.pkl')

# ７特征提取完毕，训练

In [9]:
physical_data = pd.read_pickle('../data/train_data.pkl')
index = np.load('../data/index.npy')
physical_data.index = index
# 去除inf数据
j = 0
for column in tqdm(physical_data.columns):
    new_column = []
    for i in range(len(index)):
        value = physical_data.iloc[i,j]
        if str(value) == 'inf':
            max_value = [x if str(x) not in ['inf','nan'] else 0 for x in physical_data.loc[:,column]]
            new_column.append(np.max(max_value)*1.5)
        else:
            new_column.append(value)
    physical_data[column] = new_column
    j+=1

HBox(children=(IntProgress(value=0, max=353), HTML(value='')))

In [10]:
# 裁剪数据，并生成五对训练测试集
clip = {'收缩压':[60,250],'舒张压':[40,200],'血清甘油三酯':[0.1,20],
        '血清高密度脂蛋白':[0.15,4.78],'血清低密度脂蛋白':[0.15,11.54]}
colume_name = train.columns[1::]
data_set = []
for item in colume_name:
    index = []
    y = []
    print(item)
    for i in range(len(train)):
        value = train.ix[i,item]
        try:
            c = float(value)
            if c<0:
                c = abs(c)
            if c>0:
                if c<clip[item][0]:
                    c = clip[item][0]
                elif c>clip[item][1]:
                    c = clip[item][1]
                index.append(train.ix[i,'vid'])
                y.append(np.log(c+1))
        except:
            continue
    assert len(y) == len(index)
    data_set.append((index,y))    
gc.collect()

收缩压


/home/zhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()


舒张压
血清甘油三酯
血清高密度脂蛋白
血清低密度脂蛋白


4405

In [11]:
# %%time
# # for xgb test
# temp = []
# def cal_loss(X,y,alpha):
#     # for xgb test 
#     param = {}
#     #param['objective'] = 'multi:softprob'
#     param['eta'] = 0.1  
#     param['max_depth'] = 7
#     param['silent'] = 1  
#     #param['num_class'] = 2
#     # param['scale_pos_weight'] = 3
#     param['learning_rate'] = alpha
#     param['nthread'] = 8
#     #param['colsample_bytree'] = 0.8
#     param['objective'] = 'reg:linear'
#     param['eval_metric'] = 'rmse'
#     param['alpha'] = 3
#     param['lambda'] = 1
#     param['subsample'] = 0.8
#     num_round = 5000
#     kfolds = 5
#     result = np.zeros((kfolds,len(test)))
#     i = 0
#     kf = KFold(n_splits=kfolds,shuffle=True,random_state=2017)
#     for train_index,test_index in kf.split(X):
#         X_train, X_test = X[train_index], X[test_index]
#         y_train, y_test = y[train_index], y[test_index]
#         xg_train = xgb.DMatrix(X_train, label = y_train) 
#         xg_test = xgb.DMatrix(X_test, label = y_test) 
#         watchlist = [(xg_train, 'train'), (xg_test, 'val')]
#         bst = xgb.train(param, xg_train, num_round, watchlist, early_stopping_rounds = 100)
#         y_ = bst.predict(xg_test)
#         loss = pinche(y_,y_test)
#         temp.append(loss)
#         print('loss ',loss)
# #         print(y_)
# #         print(y_test)
#         #result[i,:]=bst.predict(real_test)
#         i += 1
#         break
    
#     # result = result.mean(axis = 0)
#     return loss,bst
# def pinche(y,y_):
#     loss = 0
#     for (a,b) in zip(y,y_):
#         loss += np.square(a-b)
#     loss = loss/len(y)
#     return loss
# # for test
# loss = 0
# temp = []
# alpha = [0.03,0.03,0.01,0.03,0.03]
# droped = []
# for i in tqdm(range(5)):
# #     X = physical_data.loc[data_set[i][0],:].values
# #     y = np.array(data_set[i][1])
#     # 计算相关性
    
#     X = physical_data.loc[data_set[i][0],:]
#     y = pd.Series(data_set[i][1],index=X.index)
#     relation = []
#     drop_name = []
#     for item in X.columns:
#         dest_column = X.loc[:,item]
#         valid_index = dest_column[dest_column.isnull().values==False].index
#         p = np.corrcoef(dest_column[valid_index],y[valid_index])[0,1]
#         if str(p)=='nan':
#             drop_name.append(item)
#             continue
#         relation.append(p)
#         if abs(p)<0.001:
#             drop_name.append(item)
#     new_name = []
#     for name in X.columns:
#         if name not in drop_name:
#             new_name.append(name)
#     droped.append(len(drop_name))
#     X = X.loc[:,new_name].values
#     loss_,bst = cal_loss(X,y,alpha[i])
#     loss += loss_
    
# #     print(loss)
# #     break
# print('综合误差',loss/5)
# print(temp)
# print(droped)
#  [730,700,3000,1900,1500] 5folds
#综合误差 0.0283987430225
#[0.014185164660642451, 0.017427459798813025, 0.070259258661747953, 0.010533861670180705, 0.029587970321256648]
#[1000,800,3400,1600,1000]
#综合误差 0.0284058652739
#[0.014108574382090414, 0.017440564477319388, 0.070309427643018216, 0.010475003189642052, 0.029695756677422185]
#[911,1015,3400,2000,1000]
#综合误差 0.0279969044026 扩充了０／１数据
#[0.013857185028851752, 0.017187679117757895, 0.069640120188378729, 0.010260849633918358, 0.029038688044271885]
#[1112,942,4000,1300,1600/1500]
#综合误差 0.0282251169377
#[0.013928837700233097, 0.017835912899625734, 0.068658421347080126, 0.01077727989589701, 0.029925132845582743]
#

In [12]:
# for submit　xgb and lgb bagging
# real_test = xgb.DMatrix(physical_data.loc[test['vid'],:].values)
# param = {}
# #param['objective'] = 'multi:softprob'
# param['eta'] = 0.1  
# param['max_depth'] = 7
# param['silent'] = 1  
# #param['num_class'] = 2
# # param['scale_pos_weight'] = 3
# param['learning_rate'] = 0.03
# param['nthread'] = 7
# #param['colsample_bytree'] = 0.8
# param['objective'] = 'reg:linear'
# param['eval_metric'] = 'mae'
# param['alpha'] = 3
# param['lambda'] = 1
# param['subsample'] = 0.8

def cal_loss_2(X,y,lgb_tree,xgb_tree,learning_rate,real_test):
    param_lgb = {
    'num_leaves':128, 
    #'num_leaves':100, 
    'objective':'regression',
    'metric':'l2',
    'learning_rate': learning_rate,
    'num_threads':8,
     'subsample':0.8,
    'lambda_l1':1,
    'lambda_l2':1
    #'min_sum_hessian_in_leaf':100
    }
    param_xgb = {
        #param['objective'] = 'multi:softprob'
    'eta' : 0.1,  
    'max_depth' : 7,
    'silent' : 1,  
    'learning_rate' : learning_rate,
    'nthread' : 8,
    'objective' : 'reg:linear',
    'eval_metric' : 'rmse',
    'alpha' : 3,
    'lambda' : 1,
    'subsample': 0.8
    
    }
    num_round = n
    X_train = X
    y_train = y
    # lightgbm
    train_data = lgb.Dataset(X_train,label = y_train)
    bst = lgb.train(param_lgb, train_data, num_boost_round=lgb_tree)
    y_lgb = np.array(bst.predict(real_test))

    # xgboost
    xg_train = xgb.DMatrix(X_train, label = y_train) 
    num_round = xgb_tree
    bst = xgb.train(param_xgb, xg_train, num_round)
    y_xgb = np.array(bst.predict(xgb.DMatrix(real_test)))
    y_ = (y_lgb+y_xgb)/2
    return y_
def pinche(y,y_):
    loss = 0
    for (a,b) in zip(y,y_):
        loss += np.square(a-b)
    loss = loss/len(y)
    return loss

In [13]:
# submit
lgb_tree = [1200,1000,1700,2600,2000]
xgb_tree = [2200,2500,3400,5000,4200]
alpha = [0.01,0.01,0.01,0.01,0.01]
submit_df = pd.DataFrame()
for i in tqdm(range(5)):
#     X = physical_data.loc[data_set[i][0],:].values
#     y = np.array(data_set[i][1])
    # 计算相关性
    X = physical_data.loc[data_set[i][0],:]
    y = pd.Series(data_set[i][1],index=X.index)
    relation = []
    drop_name = []
    for item in X.columns:
        dest_column = X.loc[:,item]
        valid_index = dest_column[dest_column.isnull().values==False].index
        p = np.corrcoef(dest_column[valid_index],y[valid_index])[0,1]
        if str(p)=='nan':
            drop_name.append(item)
            continue
        relation.append(p)
        if abs(p)<0.001:
            drop_name.append(item)
    new_name = []
    for name in X.columns:
        if name not in drop_name:
            new_name.append(name)
    X = X.loc[:,new_name].values
    test_array = physical_data.loc[test['vid'],new_name].values
    result = cal_loss_2(X,y,lgb_tree[i],xgb_tree[i],alpha[i],test_array)
    submit_df[colume_name[i]] = result
print(submit_df)

# 计算结果反变换
for i in range(5):
    submit_df[colume_name[i]] = submit_df[colume_name[i]].apply(lambda x:np.exp(x)-1)

HBox(children=(IntProgress(value=0, max=5), HTML(value='')))

/home/zhu/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:1110: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/zhu/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:73: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/home/zhu/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3159: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/home/zhu/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3093: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)
/home/zhu/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3093: RuntimeWarning: invalid value encountered in multiply
  c *= 1. / np.float64(fact)
/home/zhu/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3167: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/zhu/anaconda3/lib


           收缩压       舒张压    血清甘油三酯  血清高密度脂蛋白  血清低密度脂蛋白
0     4.894293  4.416623  0.903307  0.907488  1.361741
1     4.819389  4.259623  0.741996  0.948498  1.402226
2     4.853039  4.432735  1.400970  0.820149  1.328993
3     4.854724  4.363793  0.990574  0.970505  1.140764
4     4.954130  4.363240  1.148541  0.938735  1.528076
5     4.880487  4.360287  0.900654  0.977576  1.400404
6     4.848738  4.329467  0.706969  0.959966  1.083818
7     4.925280  4.480758  1.297094  0.799793  1.227767
8     4.862428  4.414697  1.128017  0.925891  1.180996
9     4.832503  4.390817  0.920273  0.817536  1.353452
10    4.980227  4.490771  1.077023  0.883018  1.408843
11    4.864045  4.415248  2.171673  0.927813  1.375255
12    4.863795  4.344909  0.786722  0.921636  1.431399
13    4.931163  4.359260  0.885274  0.789415  1.408057
14    4.797137  4.283696  0.703125  0.931822  1.344567
15    4.882504  4.336369  1.005827  0.847260  1.347418
16    4.805615  4.282910  0.720628  0.833993  1.002671
17    4.8

# 8保存结果

In [14]:
import datetime
nowTime=datetime.datetime.now().strftime('_%Y%m%d_%H%M%S')
submit_name = '../submit/submit'+nowTime+'.csv'
submit_df.index = test['vid']
submit_df.to_csv(submit_name,header=None)